In [1]:
import initialize_notebook

import numpy as np
import pandas as pd

import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
%matplotlib inline

import io
from google.colab import files

from ams import config
from sklearn.preprocessing import StandardScaler
import pandas_datareader.data as web
import math
#import xlsxwriter
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime as dt


ModuleNotFoundError: No module named 'termios'

In [2]:
import sys
from pathlib import Path
from typing import Dict

from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, Row, StructField, DoubleType, ArrayType, StringType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from ams.config import constants
from ams.services import file_services
from ams.services import spark_service
from ams.utils import sentiment

spark = spark_service.get_or_create('learning_prep')

Setting up logging...
Will use logging path: /home/jovyan/work/data/logs/alpha_media_signal


In [3]:
twitter_folder = 'twitter'

flat_tweet_path_1 = Path(constants.DATA_PATH, twitter_folder, "sent_drop", "tweet_sent_2020-09-14_01-54-46-424.27")
flat_tweet_path_2 = Path(constants.DATA_PATH, twitter_folder, "sent_drop", "tweet_sent_2020-09-16_04-20-22-347.55")
flat_tweet_path_3 = Path(constants.DATA_PATH, twitter_folder, "sent_drop", "tweet_sent_2020-09-18_13-56-57-859.2")
flat_tweet_path_4 = Path(constants.DATA_PATH, twitter_folder, "sent_drop", "tweet_sent_2020-09-18_22-23-53-311.06")
flat_tweet_path_5 = Path(constants.DATA_PATH, twitter_folder, "sent_drop", "tweet_sent_2020-09-20_12-55-15-934.3")

In [4]:
df_1 = spark.read.csv(str(flat_tweet_path_1), header=True)
df_2 = spark.read.csv(str(flat_tweet_path_2), header=True)
df_3 = spark.read.csv(str(flat_tweet_path_3), header=True)
df_4 = spark.read.csv(str(flat_tweet_path_4), header=True)
df_5 = spark.read.csv(str(flat_tweet_path_5), header=True)

print(f'df_1: {df_1.count()}')
print(f'df_2: {df_2.count()}')
print(f'df_3: {df_3.count()}')
print(f'df_4: {df_4.count()}')
print(f'df_5: {df_5.count()}')

df_1: 2606202
df_2: 312411
df_3: 1376331
df_4: 690703
df_5: 909703


In [5]:
from functools import reduce
from pyspark.sql import DataFrame

df = reduce(DataFrame.unionByName, [df_1, df_2, df_3, df_4, df_5])

df_red = df

In [6]:
print(df_red.columns)

['entities_urls_1', 'text', 'entities_user_mentions_2', 'entities_urls_2', 'user_profile_background_tile', 'entities_urls_3', 'user_screen_name', 'favorite_count', 'in_reply_to_screen_name', 'user_verified', 'metadata_result_type', 'user_listed_count', 'user_time_zone', 'user_profile_banner_url', 'entities_urls_0', 'user_has_extended_profile', 'in_reply_to_status_id', 'user_profile_background_color', 'in_reply_to_user_id', 'user_statuses_count', 'user_url', 'user_protected', 'user_profile_background_image_url', 'user_is_translation_enabled', 'user_utc_offset', 'contributors', 'user_location', 'user_profile_text_color', 'source', 'lang', 'metadata_iso_language_code', 'entities_user_mentions_0', 'user_profile_link_color', 'user_geo_enabled', 'user_profile_image_url_https', 'user_favourites_count', 'place_country', 'place_name', 'possibly_sensitive', 'user_description', 'user_profile_sidebar_fill_color', 'user_friends_count', 'id', 'user_notifications', 'user_default_profile', 'retweeted'

In [7]:
from ams.utils import date_utils
from pyspark.sql import types as T
from datetime import datetime

twitter_format = date_utils.TWITTER_LONG_FORMAT 
def udf_wrapper(date_string: str):
    return int(datetime.strptime(date_string, twitter_format).timestamp())
        
parse_udf = F.udf(udf_wrapper, T.IntegerType())

print(f'Count: {df_red.count()}')

df_dated = df_red.withColumn('created_at_timestamp', parse_udf(F.col('created_at')))

Count: 5895350


In [8]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from ams.utils import date_utils

df_winn = df_dated.select(*["created_at",
                 "created_at_timestamp",
                 "user_screen_name",
                 "favorite_count",
                 "in_reply_to_screen_name",
                 "user_verified",
                 "metadata_result_type",
                 "user_listed_count",
                 "user_time_zone",
                 "user_has_extended_profile",
                 "in_reply_to_status_id",
                 "user_statuses_count",
                 "user_protected",
                 "user_is_translation_enabled",
                 "user_location",
                 "lang",
                 "user_geo_enabled",
                 "place_country",
                 "place_name",
                 "possibly_sensitive",
                 "user_friends_count",
                 "retweet_count",
                 "user_follow_request_sent",
                 "user_followers_count",
                 "f22_ticker",
                 "f22_has_cashtag",
                 "f22_ticker_in_text",
                 "f22_num_other_tickers_in_tweet",
                 "f22_sentiment_pos",
                 "f22_sentiment_neu",
                 "f22_sentiment_neg",
                 "f22_sentiment_compound"])

In [9]:
df_has_time = df_winn.where(F.col('created_at_timestamp').isNotNull())

df_sample = df_has_time.sample(fraction=.001)
df_sample.select(*['created_at_timestamp', 'created_at']).toPandas().head()

,created_at_timestamp,created_at
0,1598462722,Wed Aug 26 17:25:22 +0000 2020
1,1598453087,Wed Aug 26 14:44:47 +0000 2020
2,1598481623,Wed Aug 26 22:40:23 +0000 2020
3,1598485198,Wed Aug 26 23:39:58 +0000 2020
4,1598478367,Wed Aug 26 21:46:07 +0000 2020


In [18]:
df_pd = df_sample.toPandas().sort_values('created_at_timestamp', inplace=False)

# result = pd.cut(df_pd["created_at_timestamp"].values, 20, labels=range(1,21))
# df_pd['bin'] = result.codes

# df_grouped = df_pd.groupby(['bin']).agg(['count'])

# df_grouped.columns = ['_'.join(str(i) for i in col) for col in df_grouped.columns]

# df_chart = df_grouped.reset_index('bin')[["bin", "created_at_timestamp_count"]]

# df_chart.head()

# df_chart.plot(kind='bar',figsize=(10,5),legend=None)

day_groups = df_pd.groupby(df_test['date'])['date'].count()

# ['value'].count()
day_groups.plot(kind='bar',figsize=(10,5),legend=None)

# day_groups.head()

NameError: name 'df_test' is not defined

In [11]:
df_has_time.columns

['created_at',
 'created_at_timestamp',
 'user_screen_name',
 'favorite_count',
 'in_reply_to_screen_name',
 'user_verified',
 'metadata_result_type',
 'user_listed_count',
 'user_time_zone',
 'user_has_extended_profile',
 'in_reply_to_status_id',
 'user_statuses_count',
 'user_protected',
 'user_is_translation_enabled',
 'user_location',
 'lang',
 'user_geo_enabled',
 'place_country',
 'place_name',
 'possibly_sensitive',
 'user_friends_count',
 'retweet_count',
 'user_follow_request_sent',
 'user_followers_count',
 'f22_ticker',
 'f22_has_cashtag',
 'f22_ticker_in_text',
 'f22_num_other_tickers_in_tweet',
 'f22_sentiment_pos',
 'f22_sentiment_neu',
 'f22_sentiment_neg',
 'f22_sentiment_compound']

In [12]:
df_sample.select(*['user_followers_count', 'f22_sentiment_compound']).toPandas().head()

,user_followers_count,f22_sentiment_compound
0,143,0.4199
1,915,0.4939
2,3222,0.6705
3,601,0.3818
4,667,-0.4404


In [13]:
df_time_converted = df_has_time.withColumn('user_followers_count', F.col('user_followers_count').cast(T.IntegerType())).withColumn('f22_sentiment_compound', F.col('f22_sentiment_compound').cast(T.FloatType()))

df_has_scores = df_time_converted.where(F.col('user_followers_count') > 0).where(F.col('f22_sentiment_compound') != 0)

print(f'Has followers and compound score: {df_has_scores.count()}')

df_com_score = df_has_scores.withColumn('f22_compound_score', F.col('user_followers_count') * F.col('f22_sentiment_compound'))

Has followers and compound score: 3652061


In [14]:
from ams.utils import date_utils
from datetime import datetime

standard_format = date_utils.STANDARD_DAY_FORMAT 
def convert_to_date_string(epoch_millis: str):
    dt = datetime.fromtimestamp(int(epoch_millis))
    return dt.strftime(standard_format)
        
parse_udf = F.udf(convert_to_date_string)

df_dated = df_com_score.withColumn('date', parse_udf(F.col('created_at_timestamp')))

In [17]:
from ams.services import dataframe_services

tag = 'learning_prep_drop'

sent_drop_path = Path(constants.DATA_PATH, twitter_folder, tag)

dataframe_services.persist_dataframe_as_csv(df=df_dated, output_drop_folder_path=sent_drop_path, prefix=tag, num_output_files=1)

/home/jovyan/work/data/twitter/learning_prep_drop/learning_prep_drop_2020-09-20_13-29-52-218.1
